In [ ]:
#Reading in the Neccesary Packages
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
#Reading in the Neccesary Data (2016-2020 NBA Individual Player Stats), All Data Came From Basketball Reference 
#Advanced Stats
#2020
Player2020 = pd.read_csv("data/2020Advanced.csv")
new20 = Player2020.Player.str.split("\\", expand=True)
Player2020['Player'] = new20[0]
Player2020['PlayerID'] = new20[1]
Player2020['Season'] = 2020
Player2020
#2019
Player2019 = pd.read_csv("data/2019Advanced.csv")
new19 = Player2019.Player.str.split("\\", expand=True)
Player2019['Player'] = new19[0]
Player2019['PlayerID'] = new19[1]
Player2019['Season'] = 2019
Player2019
#2018
Player2018 = pd.read_csv("data/2018Advanced.csv")
new18 = Player2018.Player.str.split("\\", expand=True)
Player2018['Player'] = new18[0]
Player2018['PlayerID'] = new18[1]
Player2018['Season'] = 2018
Player2018
#2017
Player2017 = pd.read_csv("data/2017Advanced.csv")
new17 = Player2017.Player.str.split("\\", expand=True)
Player2017['Player'] = new17[0]
Player2017['PlayerID'] = new17[1]
Player2017['Season'] = 2017
Player2017
#2016
Player2016 = pd.read_csv("data/2016Advanced.csv")
new16 = Player2016.Player.str.split("\\", expand=True)
Player2016['Player'] = new16[0]
Player2016['PlayerID'] = new16[1]
Player2016['Season'] = 2016
Player2016

In [ ]:
#Attempting to Scrape All Total Stats from 2016-2020
def scrape_season(season):
    url = "https://www.basketball-reference.com/leagues/NBA_" + str(season) + "_totals.html"
    print(url)
    html = urlopen(url)
    bs = BeautifulSoup(html.read(), 'lxml')
    tables = bs.find_all("table", {'class':'sortable'})
    stats_tables = pd.read_html(str(tables))
    
    total_stats = stats_tables[0].copy()
    total_stats['Season'] = season
    
    return(total_stats)

import time
all_total_stats = pd.DataFrame()

seasons = np.arange(2016,2021)
for season in seasons:
    total_stats = scrape_season(season)
    all_total_stats = pd.concat([total_stats, all_total_stats])
    time.sleep(np.random.uniform(2,5))
    
all_total_stats

In [ ]:
#Had an Issue Joining the Total Scraped Tables and Advanced CSVs, So Wanted to Try Getting Totals CSVs
#Total Stats
#2020
Total2020 = pd.read_csv("data/2020Totals.csv")
newTotal20 = Total2020.Player.str.split("\\", expand=True)
Total2020['Player'] = newTotal20[0]
Total2020['PlayerID'] = newTotal20[1]
Total2020
#2019
Total2019 = pd.read_csv("data/2019Totals.csv")
newTotal19 = Total2019.Player.str.split("\\", expand=True)
Total2019['Player'] = newTotal19[0]
Total2019['PlayerID'] = newTotal19[1]
Total2019
#2018
Total2018 = pd.read_csv("data/2018Totals.csv")
newTotal18 = Total2018.Player.str.split("\\", expand=True)
Total2018['Player'] = newTotal18[0]
Total2018['PlayerID'] = newTotal18[1]
Total2018
#2017
Total2017 = pd.read_csv("data/2017Totals.csv")
newTotal17 = Total2017.Player.str.split("\\", expand=True)
Total2017['Player'] = newTotal17[0]
Total2017['PlayerID'] = newTotal17[1]
Total2017
#2016
Total2016 = pd.read_csv("data/2016Totals.csv")
newTotal16 = Total2016.Player.str.split("\\", expand=True)
Total2016['Player'] = newTotal16[0]
Total2016['PlayerID'] = newTotal16[1]
Total2016

In [ ]:
#Joining the Advanced Stats With Total (Box Score/Traditional) Stats
#2020
All2020 = pd.merge(Total2020, Player2020, how = 'inner', on = (['Player', 'Tm']))
All2020
#2019
All2019 = pd.merge(Total2019, Player2019, how = 'inner', on = (['Player', 'Tm']))
All2019
#2018
All2018 = pd.merge(Total2018, Player2018, how = 'inner', on = (['Player', 'Tm']))
All2018
#2017
All2017 = pd.merge(Total2017, Player2017, how = 'inner', on = (['Player', 'Tm']))
All2017
#2016
All2016 = pd.merge(Total2016, Player2016, how = 'inner', on = (['Player', 'Tm']))
All2016

In [ ]:
#Stacking All of the Data Sets Together
AllPlayers = pd.concat([All2020, All2019, All2018, All2017, All2016])
AllPlayers
AllPlayers = AllPlayers.set_index('Player')
AllPlayers

In [ ]:
#Dropping Other Advanced Stats and Unneeded Columns
AllPlayers = AllPlayers.drop(['Rk', 'Pos_x', 'Age_x', 'Tm', 'G_x', 'GS', 'MP_x', 'PlayerID_x', 'Pos_y', 'Age_y', 'G_y', 'PlayerID_y', 'Rk▲'], axis=1)
AllPlayers

In [ ]:
#Writing Final Dataset to CSV
AllPlayers.to_csv("data/AllPlayers.csv")

In [ ]:
#Boxplot of BPM Over Last 5 Years
AllPlayers.boxplot(column = 'BPM')

In [ ]:
#Scatter Plot Matrix
AllPlayers_selected_data = AllPlayers[['FG', 'FT', 'TOV', 'PF']]
AllPlayers_selected_data

sns.pairplot(AllPlayers_selected_data, diag_kind = "kde", kind = "reg", 
             plot_kws={'line_kws':{'color':'red'}})

In [ ]:
#Creating the Model
bpm_ols_formula = "BPM ~ FG + Q('FG%') + Q('3P') + Q('3P%') + FT + Q('FT%') + ORB + DRB + AST + STL + BLK + TOV + PF" 
bpm_ols_model = smf.ols(formula = bpm_ols_formula, data = AllPlayers).fit()
bpm_ols_model.summary()

In [ ]:
AllPlayers['Players'] = AllPlayers.index
AllPlayers

In [ ]:
#Plotting Residuals vs Fitted Values
residuals = bpm_ols_model.resid
fitted = bpm_ols_model.fittedvalues
top2 = abs(residuals).sort_values(ascending = False)[:2]

fig, ax = plt.subplots()
ax.scatter(fitted, residuals, edgecolors="k", facecolors="none")
ax.set_ylabel("Residual")
ax.set_xlabel("Fitted Values")
ax.plot([min(fitted),max(fitted)], [0,0], color="k", linestyle=":", alpha = 0.3)

for i in top2.index:
    ax.annotate(str(AllPlayers.Players[i]) + " " + str(AllPlayers.Players[i]), xy=(fitted[i], residuals[i] +0.002), size=9)    